In [ ]:
from pathlib import Path
from tqdm import tqdm
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import folium
from shapely.geometry import Polygon, MultiPolygon, Point
from random import shuffle
import cartopy.crs as ccrs
from matplotlib import colors, ticker


from src.plotting.util import plot_gdf_column, load_grids

import warnings
warnings.filterwarnings("ignore")  # hide every warning

# --- Configuration ---

In [ ]:
BASE = Path("/Users/kyledorman/data/planet_coverage/points_30km/")
SHORELINES = BASE.parent / "shorelines"

In [ ]:
orig_grids_df = gpd.read_file(SHORELINES / "coastal_grids.gpkg")

orig_grids_df.head(3)

In [ ]:
cents = orig_grids_df.copy()
cents.geometry = cents.geometry.centroid
cents = cents.to_crs("EPSG:4326")

grids_df = orig_grids_df[
    (~orig_grids_df.is_land) &
    (orig_grids_df.dist_km < 20) &
    (cents.geometry.centroid.y > -81.5) & 
    (cents.geometry.centroid.y < 81.5)
].copy()

len(grids_df)

In [ ]:
merged = grids_df.union_all()
len(merged.geoms)

In [ ]:
from random import shuffle
import cartopy.crs as ccrs
from matplotlib import colors, ticker

mdf = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=grids_df.crs)

ids = list(range(len(mdf)))
shuffle(ids)
mdf["id"] = ids

n_ids  = mdf["id"].nunique()
base_cmap = plt.get_cmap("tab20", n_ids)  # up to 20 unique colours
cmap      = colors.ListedColormap(base_cmap(range(n_ids)))
norm      = colors.BoundaryNorm(range(n_ids + 1), n_ids)

# Pick any Cartopy projection
proj = ccrs.Sinusoidal()           # or ccrs.Mollweide(), ccrs.Robinson(), …

fig = plt.figure(figsize=(12, 6))
ax  = plt.axes(projection=proj)
ax.set_global()

# Re-project your data on the fly with `transform`
mdf.plot(
    column="id",
    ax=ax,
    cmap=cmap,
    norm=norm,
    linewidth=0.15,
    edgecolor="black",
    transform=ccrs.Sinusoidal(),   # <- incoming lon/lat coords
)
plt.tight_layout()
plt.show()

In [ ]:
mdf.geometry = mdf.geometry.simplify(50, preserve_topology=True)
mdf["garea"] = mdf.geometry.area
mdf = mdf.sort_values(by="garea", ascending=False)

In [ ]:
from src.gen_points_map import compute_step, make_equal_area_hex_grid

robinson_crs = "ESRI:54030"
sinus_crs = "ESRI:54008"
hex_size: float = 1.0
cell_size_m = compute_step(hex_size)

grids = []
offsets = []
for offset_x in range(0, 8):
    for offset_y in range(0, 4):
        offsets.append((offset_x * cell_size_m / 8 , offset_y * cell_size_m / 8))

for offset_x, offset_y in tqdm(offsets):
    hex_grid = make_equal_area_hex_grid(cell_size_m, robinson_crs, offset_x, offset_y).rename(columns={"cell_id": "hex_id"})
    inter = gpd.sjoin(hex_grid.to_crs(sinus_crs), grids_df)
    counts = inter[["hex_id", "cell_id"]].groupby("hex_id").count()
    hex_ids = counts[counts.cell_id > 10].index
    c = len(counts)
    grids.append((offset_x, offset_y, c, hex_grid))

In [ ]:
best = pd.DataFrame([a[:3] for a in grids], columns=["x", "y", "ic"])
best = best.sort_values("ic")
best

In [ ]:
best.iloc[3]

In [ ]:
hex_grid = make_equal_area_hex_grid(cell_size_m, robinson_crs, best.x.iloc[3], best.y.iloc[3]).rename(columns={"cell_id": "hex_id"})
inter = gpd.sjoin(hex_grid.to_crs(sinus_crs), grids_df)
counts = inter[["hex_id", "cell_id"]].groupby("hex_id").count()
hex_ids = counts[counts.cell_id > 10].index
hgb = hex_grid.set_index("hex_id").loc[hex_ids]

In [ ]:
hex_grid.to_file("/Users/kyledorman/data/planet_coverage/shorelines/hex_grids.gpkg

In [ ]:
ids = list(range(len(hgb)))
shuffle(ids)
hgb["id"] = ids

n_ids  = hgb["id"].nunique()
base_cmap = plt.get_cmap("tab20", n_ids)  # up to 20 unique colours
cmap      = colors.ListedColormap(base_cmap(range(n_ids)))
norm      = colors.BoundaryNorm(range(n_ids + 1), n_ids)

# Pick any Cartopy projection
proj = ccrs.Robinson()           # or ccrs.Mollweide(), ccrs.Robinson(), …

fig = plt.figure(figsize=(12, 6))
ax  = plt.axes(projection=proj)
ax.set_global()

# Re-project your data on the fly with `transform`
hgb.plot(
    column="id",
    ax=ax,
    cmap=cmap,
    norm=norm,
    linewidth=0.05,
    edgecolor="black",
    transform=ccrs.Robinson(),   # <- incoming lon/lat coords
)
import cartopy
# ax.add_feature(cartopy.feature.OCEAN, zorder=0)  # type: ignore
# ax.add_feature(cartopy.feature.LAND, zorder=0)  # type: ignore
ax.coastlines(resolution="110m", linewidth=0.3, zorder=0)  # type: ignore
plt.tight_layout()
plt.savefig("/Users/kyledorman/Desktop/1_5deg_hex.png")
plt.show()

In [ ]:
mldf = gpd.read_file(SHORELINES / "mainlands.gpkg")
bidf = gpd.read_file(SHORELINES / "big_islands.gpkg")
sidf = gpd.read_file(SHORELINES / "small_islands.gpkg")

In [ ]:
box_grid.intersects(invalid_region.to_crs(sinus).geometry).sum()

In [ ]:
centroid = invalid_region.geometry.iloc[0].centroid
base_map = folium.Map(location=[centroid.y, centroid.x], zoom_start=5, width=1000, height=800)

mldf2 = gpd.read_file(SHORELINES / "mainlands.gpkg")
inv_mldf = mldf2[mldf2.intersects(invalid_region.geometry.iloc[0])]

for _,  row in invalid_region.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda feature: {
            "color": "blue",
            "weight": 1,
        }
    ).add_to(base_map)

for idx, row in inv_mldf.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda feature: {
            "color": "red",
            "weight": 1,
        }
    ).add_to(base_map)

base_map

In [ ]:
pre_simplify_tol = 50
sinus = "ESRI:54008"
wgs = "EPSG:4326"

In [ ]:
from src.gen_points_map import compute_step, make_equal_area_grid

cell_size_m = compute_step(0.05)
box_grid = make_equal_area_grid(cell_size_m, sinus)

In [ ]:
mldf.geometry = mldf.buffer(0)
bidf.geometry = bidf.buffer(0)
sidf.geometry = sidf.buffer(0)

In [ ]:
sidf_s = sidf.to_crs(sinus)

In [ ]:
bidf_s = bidf.to_crs(sinus)

In [ ]:
mldf_s = mldf.to_crs(sinus)

In [ ]:
sidf_s.geometry = sidf_s.geometry.simplify(pre_simplify_tol, preserve_topology=True)
sidf_s.is_valid.all()

In [ ]:
bidf_s.geometry = bidf_s.geometry.simplify(pre_simplify_tol, preserve_topology=True)
bidf_s.is_valid.all()

In [ ]:
invalid = ~bidf_s.geometry.is_valid
bidf_s.loc[invalid, "geometry"] = bidf_s.geometry[invalid].buffer(0)
bidf_s.is_valid.all()

In [ ]:
mldf_s.geometry = mldf_s.geometry.simplify(pre_simplify_tol, preserve_topology=True)
mldf_s.is_valid.all()

In [ ]:
land_union = gpd.GeoSeries(
    pd.concat([mldf_s.geometry, bidf_s.geometry], ignore_index=True),
    crs=sinus,
).union_all()

In [ ]:
land_union.is_valid

In [ ]:
ca_only = gpd.read_file(SHORELINES / "oregon.geojson").to_crs(sinus)
ca_only.is_valid

In [ ]:
ca_land = land_union.intersection(ca_only.iloc[0])

ca_land.geometry

In [ ]:
simp_land_union = land_union.simplify(500, preserve_topology=True)
simp_land_union.is_valid

In [ ]:
invalids = []
valids = []
for poly in simp_land_union.geoms:
    if not poly.is_valid:
        invalids.append(poly)
    else:
        valids.append(poly)

invalid_df = gpd.GeoDataFrame(geometry=invalids, crs=sinus)
invalid_df.geometry = invalid_df.geometry.buffer(0)

simp_land_union_clean = MultiPolygon(valids + invalid_df.geometry.tolist()).buffer(0)
simp_land_union_clean.is_valid

In [ ]:
type(simp_land_union_clean)

In [ ]:
keep = []
for poly in simp_land_union_clean.geoms:
    if poly.intersects(ca_only.iloc[0][0]):
        keep.append(poly)

ca_land_union = MultiPolygon(keep)

In [ ]:
from shapely import segmentize, get_coordinates
get_coordinates(poly.boundary.segmentize(100)).shape

In [ ]:
ca_grids = box_grid[box_grid.geometry.within(ca_only.iloc[0][0])]

len(ca_grids)

In [ ]:
coast_pts = []
for poly in ca_land_union.geoms:
    coast_pts.append(get_coordinates(poly.boundary.segmentize(1000)))
coast_xy = np.concatenate(coast_pts)

coast_xy.shape

In [ ]:
from scipy.spatial import cKDTree
tree = cKDTree(coast_xy)

In [ ]:
centers = ca_grids.geometry.centroid
cent_xy = np.column_stack((centers.x.values, centers.y.values))  # type: ignore

In [ ]:
dists, _ = tree.query(cent_xy, workers=-1)  # parallel threads, SciPy ≥1.9

In [ ]:
ca_grids["dist_km"] = dists / 1000

In [ ]:
ca_grids.dist_km.min(), ca_grids.dist_km.max()

In [ ]:
(ca_grids.dist_km < 100).sum()

In [ ]:
cxy = [Point(*xy) for xy in coast_xy]

In [ ]:
from branca.colormap import linear
from folium.plugins import MarkerCluster

color_scale = linear.viridis.scale(0, 100)
cent = ca_only.to_crs(wgs).iloc[0][0].centroid

m = folium.Map(
    location=[cent.y, cent.x], 
    zoom_start=5, 
    tiles="CartoDB positron",
    width=1000,
    height=800
)

for _, row in ca_grids[(ca_grids.dist_km < 40) & (ca_grids.dist_km > 20)].to_crs(wgs).iterrows():
    value = row["dist_km"]
    geom = row.geometry
    folium.GeoJson(
        data=geom,
        style_function=lambda f, col=color_scale(value): {
            "fillColor": col,
            "color":     col,      # outline same as fill
            "weight":    1,
            "fillOpacity": 0.7,
        },
        tooltip=f"{row.cell_id}: {value:0.1f}",
    ).add_to(m)


# pts = gpd.GeoDataFrame(geometry=cxy, crs=sinus).to_crs(wgs)
# for _, row in pts.iterrows():
#     folium.CircleMarker(
#         location=[row.geometry.y, row.geometry.x],
#         radius=2,
#         fill=True,
#         fill_opacity=0.6,
#         color=None,
#         fill_color="red",
#     ).add_to(m)

color_scale.add_to(m)

m